# Theory -1

## Attention Mechanisms

**1. What is the function of Q,K,V in Attention**

Consider the sentence:
**The bank of the river was muddy**


**The "Search" (Query)** : The word **"bank"** sends out a Query: "I am a noun, but I need context. Am I related to money or geography? Look for words that describe my surroundings."

**The "Match" (Key comparison)**:
The Query from "bank" compares itself against the Keys of every other word:

    "The": Key says "I am a generic article." → Match: Low.
    "River": Key says "I am a body of water, geography, nature." → Match: Very High.
    "Muddy": Key says "I am a texture, dirt, related to earth." → Match: High.

**The "Weights" (Attention Scores) : Softmax**
The **"River"** Key matches the **"geographic"** part of the **"Bank"** Query perfectly. Because of this high match, the model assigns a high **```Attention Score (e.g., 0.85)```** to the word "river." and low sore to rest of the words

**The "Retrieval" (Value merging)**:
The model now takes all the Value (the actual data/meaning) of each word and multiplies them by their scores. The controbution of **"river"** in determining the nature of **"bank"** is so high rest can be ignored.

**The "New Meaning" (Context Awareness)**
By the end of this layer, the mathematical vector for "bank" is no longer just the generic word. It has been "pulled" toward the meaning of "river."

But for this sentence **The bank closed my account** Query from **"bank"** would have found a high match with the Key for **"account"** instead.

**2. Multi-Head Attention vs Single-Head Attention**

**Why multiple heads?** Each head learns different relationship patterns in separate representation subspaces. Head 1 might capture syntax, Head 2 semantics, Head 3 long-range dependencies.

**Math:** Instead of one $d_{model} \times d_{model}$ attention, $d_{model}$ split into $h$ heads of size $d_k = d_{model}/h$

Final output: $\text{Concat}(head_1, ..., head_h)W^O$

**Scaling factor:** $\frac{1}{\sqrt{d_k}}$ prevents dot products from growing too large, which would push softmax into saturation regions with tiny gradients.

***

**3. Multi-Query Attention (MQA) vs Multi-Head Attention (MHA)**

**Why share K/V?** In MHA, each head has separate $W^K, W^V$ matrices. MQA uses **one shared K/V projection** for all heads, only queries stay separate.

**Memory savings:** KV-cache size drops from $2 \times L \times h \times d_k \times n$ to $2 \times L \times d_k \times n$ (~8x reduction for 8 heads).

$2$ -> both ```Keys & Values``` vector is stored for each token

$L$ -> no. of ```Layers``` in transformer (32 layers in Llama-2-7B)

$h$ -> ```Heads``` each attention head has its own unique K and V projections

$d_k$ -> ```Head Dimension``` for single head in **MHA**

$n$  -> ```Sequence Length```: no. of tokens currently in context(prompt + generated tokens)

**Trade-off:** Slight quality drop (~1-2% on benchmarks) but 4-5x faster inference due to reduced memory bandwidth.

***

**4. Grouped Query Attention (GQA)**

**Why groups?** Middle ground between MHA (h separate KV pairs) and MQA (1 shared KV pair). Use $g$ groups, e.g., 8 heads share 2 KV pairs (4 heads per group).

**Math:** If $h=8, g=2$

heads 1-4 share KV₁, heads 5-8 share KV₂.

**Optimal:** LLaMA 2 70B uses $h=64, g=8$ (8 heads per group) for balance.

***

### Flash Attention
**Why $O(n^2)$ memory?** Standard attention computes full $n \times n$ attention matrix before softmax.

**Fix:** Tile the computation into blocks, load from ```HBM → SRAM → compute → write back```. Never materialize full matrix.

**Result:** Memory drops from $O(n^2)$ to $O(n)$ while being 2-4x faster via kernel fusion.

***

### Sliding Window Attention
**Why limit window?** Mistral uses 4096-token sliding window. Each token only attends to previous 4096 tokens, not full sequence.

**Math:** Attention mask $M_{ij} = 0$ if $i - j > 4096$, else 1.

**Gradient flow:** Still $O(n^2)$ theoretically, but practical memory and compute are $O(n \cdot w)$ where $w=4096$.

**Loss:** Can't directly reference tokens >4096 positions back, but multi-layer stacking allows indirect access (layer 2 sees layer 1's window, so effectively 2× window depth).

***

## Positional Encodings

### Sinusoidal vs Learned Embeddings
**Sinusoidal formula:**

$PE_{(pos, 2i)} = \sin(pos/10000^{2i/d})$,
$PE_{(pos, 2i+1)} = \cos(pos/10000^{2i/d})$.

**Why?** Allows extrapolation: $PE_{pos+k}$ can be expressed as linear function of $PE_{pos}$

**Learned:** Train $W_{pos} \in \mathbb{R}^{max\_len \times d}$. Fails at test-time lengths > training max_len.

**Modern choice:** Neither. Use RoPE or Alibi instead.

***

### Rotary Position Embedding (RoPE)
**Core idea:** Rotate query/key vectors by angle proportional to position.

**Math:** For position $m$, apply rotation matrix $R_m$:
$$
f(q, m) = \begin{bmatrix} \cos(m\theta) & -\sin(m\theta) \\ \sin(m\theta) & \cos(m\theta) \end{bmatrix} q
$$

**Key property:** $q_m^T k_n = (R_m q)^T (R_n k) = q^T R_{n-m} k$

dot product depends only on **relative distance** $n-m$

**Used in:** LLaMA, GPT-NeoX, PaLM.

***

### Alibi (Attention with Linear Biases)
**Formula:** Add bias $-m \cdot |i-j|$ to attention scores, where $m$ is head-specific slope.

**Math:** $\text{softmax}(QK^T + \text{Alibi})$,

Alibi matrix is $[-m, -2m, -3m, ...]$ for each row.

**Advantage:** No positional embeddings needed. Extrapolates better to longer sequences than RoPE in some tasks.

**Used in:** BLOOM, MPT.

***

## Normalization

### Pre-Norm vs Post-Norm
**Post-norm (original Transformer):**
$x = \text{LN}(x + \text{Attention}(x))$

**Pre-norm (modern):**
$x = x + \text{Attention}(\text{LN}(x))$

**Why pre-norm?** Gradients flow directly through residual path without passing through normalization. Stabilizes training in deep models (>24 layers). GPT-2/3, LLaMA all use pre-norm.

***

### RMSNorm vs LayerNorm
**LayerNorm:**
$y = \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} \cdot \gamma + \beta$

**RMSNorm (removes mean centering):**
$y = \frac{x}{\sqrt{\frac{1}{d}\sum x_i^2 + \epsilon}} \cdot \gamma$

**Speedup:** 10-30% faster (single pass vs two-pass), less memory.

**Why it works:** Re-centering contributes little to transformer performance; rescaling alone suffices.[5]

***

## Activation Functions

### SwiGLU vs GELU
**GELU:** $\text{GELU}(x) = x \cdot \Phi(x)$ where $\Phi$ is Gaussian CDF.

**SwiGLU:**
$
\text{SwiGLU}(x, W, V) = (Wx \odot \sigma(Vx))
$

where $\sigma$ is Swish ($x \cdot \text{sigmoid}(x)$), $\odot$ is element-wise product.

**Why gating?** The $\odot$ operation acts as a learned filter—decides which activations to pass through.

**Performance:** LLaMA uses SwiGLU, shows ~1% improvement over GELU.

***

### Squared ReLU
**Formula:** $\text{ReLU}^2(x) = \max(0, x)^2$.

**Gradient:** $\frac{d}{dx} = 2x$ for $x > 0$, stronger gradient signal than linear ReLU.

**Usage:** Some Primer models, less common than SwiGLU.

***

## Feed-Forward Networks

### FFN Expansion Ratio
**Standard:** $d_{ff} = 4 \times d_{model}$ (e.g., 768 → 3072 in BERT).

**LLaMA:** Uses $8 \times d_{model}$ (4096 → 32768 for 7B model).

**Why?** More parameters in FFN = better memorization capacity. Trade-off: 2× FLOPs per layer.

**Parameter split:** ```In transformers, ~⅔ of parameters are in FFN layers.```

***

### Mixture of Experts (MoE)
**Formula:**
$
y = \sum_{i=1}^{N} G(x)_i \cdot E_i(x)
$

where $G(x) = \text{TopK}(\text{softmax}(Wx))$ is gating network, $E_i$ are expert FFNs.

**Example:** Mixtral 8×7B has 8 experts, activates top-2 per token. Total params: 47B, active params per token: 13B.

**Load balancing loss:**
$
L_{aux} = \alpha \sum_{i=1}^{N} f_i \cdot P_i
$

where $f_i$ = fraction of tokens to expert $i$, $P_i$ = avg gate probability for expert $i$. Prevents all tokens routing to one expert.

***

## Tokenization

### BPE vs WordPiece
**BPE:** Greedy merging of most frequent byte pairs. ```"hugging" → "hug" + "ging"```.

**WordPiece:** Maximizes likelihood $P(corpus)$ when choosing merges instead of raw frequency.

**Vocab size:** GPT-2 uses 50k BPE, BERT uses 30k WordPiece.

**Compression:** ```Larger vocab → shorter sequences → faster training, but larger embedding matrix.```

***

### Byte-Level BPE
**Change:** Operate on UTF-8 bytes (256 base tokens) instead of Unicode characters.

**Benefit:** **Zero UNK tokens**—any text can be encoded (even emojis, rare scripts).

**Used in:** GPT-2, GPT-3, LLaMA.

***

## Architecture Variants

### Decoder-Only vs Encoder-Decoder
**Decoder-only (GPT):** Causal attention mask (can't see future). Autoregressive generation.

**Encoder-decoder (T5):** Encoder uses bidirectional attention, decoder uses causal. Better for seq2seq (translation).

**Why decoder-only dominates?** Simpler architecture, scales better, instruction-following emerges naturally with causal LM objective.

***

## Training Techniques

### Gradient Checkpointing
**Standard:** Store all activations during forward pass (memory = $O(L \cdot B \cdot d)$).

**Checkpointing:** Store only every $k$-th layer, recompute intermediate activations during backward.

**Trade-off:** ~30% slower training, but 10× memory reduction. Enables larger batch sizes.

***

### BFloat16 vs Float16
**Float16:** 1 sign, 5 exp, 10 mantissa. Range: $\pm 6.5 \times 10^4$.

**BFloat16:** 1 sign, **8 exp**, 7 mantissa. Range: $\pm 3.4 \times 10^{38}$ (same as FP32).

**Why BF16?** No loss scaling needed (wide exponent range). Used in TPUs, A100 GPUs.

***

### Gradient Accumulation
**Formula:** Accumulate gradients over $N$ microbatches before optimizer step.

**Effective batch:** $B_{eff} = N \times B_{micro}$.

**Memory:** Only $B_{micro}$ activations in memory at once, but same gradient as $B_{eff}$.

**Drawback:** BatchNorm breaks (but transformers use LayerNorm, so irrelevant).

***

## Fine-Tuning Methods

### LoRA
**Formula:** Freeze $W \in \mathbb{R}^{d \times k}$, train $\Delta W = BA$

where $B \in \mathbb{R}^{d \times r}, A \in \mathbb{R}^{r \times k}$

**Forward:** $h = Wx + BAx$

**Parameters:** Reduce from $d \times k$ to $r(d+k)$.

For $r=8, d=4096, k=4096$: 16M → 65k (250× reduction).

**Why low-rank?** Hypothesis: weight updates during fine-tuning lie in low-dimensional subspace.[9][10]

***

### QLoRA
**Change:** Load base model in 4-bit (NF4 quantization), train LoRA adapters in FP16.

**Memory:** 7B model fits in 5GB VRAM (vs 28GB for FP32).

**Double quantization:** Quantize the quantization constants themselves (saves 0.4 bits per param).

***

### Prefix Tuning vs LoRA
**Prefix tuning:** Prepend $k$ learnable vectors to K/V at each layer.

**Parameters:** $2 \times L \times k \times d$ (e.g., 2 × 24 × 20 × 768 = 737k for BERT).

**LoRA:** Modifies weight matrices directly, more parameter-efficient for same performance.

***

## Inference Optimization

### KV-Cache
**Without cache:** Generate token $t$, must recompute attention for tokens $1...t-1$.

**With cache:** Store $K_{1:t-1}, V_{1:t-1}$, only compute $K_t, V_t$.

Attention: $Q_t \cdot [K_{1:t}]$.

**Memory:** $2 \times L \times h \times d_k \times n$ floats. For LLaMA 7B at 2048 tokens: ~2GB.

***

### Speculative Decoding
**Idea:** Small model (160M) generates $$k$$ tokens speculatively, large model (7B) verifies in parallel.

**Acceptance:** If $P_{large}(x_i) \geq P_{small}(x_i)$, accept. Else, resample.

**Speedup:** 2-3× for greedy decoding when draft model accuracy >70%.

***

### GPTQ Quantization
**Method:** Use second-order Hessian information to minimize quantization error layer-by-layer.

**Formula:** Minimize $||WX - \hat{W}X||^2_F$ subject to $\hat{W}$ being quantized.

**Result:** 4-bit quantization with < 1% perplexity increase.

***


### GPTQ Quantization
**Method:** Use second-order Hessian information to minimize quantization error layer-by-layer.

**Formula:** Minimize $||WX - \hat{W}X||^2_F$ subject to $\hat{W}$ being quantized.

**Result:** 4-bit quantization with < 1% perplexity increase.

***

## Loss Functions

### Cross-Entropy with Label Smoothing
**Standard:** $L = -\log p_{true}$.

**Smoothed:** $L = -(1-\epsilon)\log p_{true} - \epsilon \sum_{i \neq true} \log p_i / (C-1)$.

**Effect:** Prevent overconfident predictions (softmax → 1.0), regularizes model.

***

### Next Token Prediction vs MLM
**Causal LM:** $p(x_t | x_{1:t-1})$. Must predict sequentially.

**MLM (BERT):** $p(x_t | x_{\neq t})$. Can see full context, but can't generate autoregressively.

**Why causal wins:** Unified pretraining + generation. Instruction-following is just conditional generation.

***

# Theory -2


## Weight Initialization

### Xavier/Glorot vs He Initialization
**Xavier (Glorot):** $W \sim \mathcal{U}\left[-\sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}, \sqrt{\frac{6}{n_{\text{in}} + n_{\text{out}}}}\right]$

**He:** $W \sim \mathcal{N}\left(0, \sqrt{\frac{2}{n_{\text{in}}}}\right)$

**Key difference:** Xavier designed for tanh/sigmoid (symmetric around 0). He for ReLU, which zeros out half the gradients—hence factor of 2 instead of 6 to compensate.

**Rule of thumb:** Use Xavier for attention projections (mostly linear). Use He for FFN layers (followed by ReLU/GELU).

**Embeddings:** Typically $\mathcal{N}(0, 0.02^2)$ — small enough to avoid saturation early in training.

---

### LayerNorm: Why γ=1, β=0?
**At init:** $\text{LN}(x) = \frac{x - \mu}{\sigma}$ (pure normalization, no learned transform)

**During training:** Network learns optimal $\gamma$ (scale) and $\beta$ (shift) per dimension.

**Why start at identity?** Stable gradients from step 1. Network decides what rescaling it needs, rather than fighting random initial scales.

---

## Attention Mechanics

### Softmax Temperature Scaling
**Standard:** $\text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)$

**With temperature:** $\text{softmax}\left(\frac{QK^T}{T \cdot \sqrt{d_k}}\right)$

**Effect:**
- $T > 1$: Flatter distribution → more token diversity
- $T < 1$: Sharper peaks → focus on top matches
- Some models learn $T$ per head (init ~0.5)

**Mental model:** Temperature is like "attention confidence." High T = "I'm uncertain, consider everything." Low T = "I know what's important."

---

### Causal Masking
**Implementation:** Set $M_{ij} = -\infty$ where $j > i$ before softmax

**Result:** $\text{softmax}([-\infty, -\infty, 2.1, 3.5]) = [0, 0, 0.25, 0.75]$

**Why at inference?** Even though generation is sequential, keeping the mask ensures training/inference consistency—same attention patterns.

**Common bug:** Forgetting to mask during fine-tuning leads to subtle distribution shifts.

---

### Attention vs Residual Dropout
**Attention dropout:** Applied to weights: $\text{Dropout}(\text{softmax}(QK^T))$
- Randomly zeros some attention connections

**Residual dropout:** Applied after attention: $x + \text{Dropout}(\text{Attention}(x))$
- Stochastically drops entire sublayer contributions

**Modern trend:** Large models (>10B params) often skip dropout entirely. Scale provides regularization.

---

## Model Parallelism

### Three Parallelism Types
**Data Parallel (DP):**
- Clone full model on N GPUs
- Each GPU processes different batch slice
- All-reduce gradients once per step
- **Bottleneck:** Batch size

**Pipeline Parallel (PP):**
- Vertical split: layers 0-11 on GPU0, 12-23 on GPU1
- Forward/backward passes bubble through pipeline
- **Bottleneck:** Pipeline bubble time (~12% wasted)

**Tensor Parallel (TP):**
- Horizontal split: $Y = X[W_1|W_2]$ → two GPUs compute $XW_1$ and $XW_2$ in parallel
- All-gather results per layer
- **Bottleneck:** All-gather bandwidth

**In practice:** GPT-3 uses DP × PP × TP (3D parallelism). 8-way DP, 16-way PP, 8-way TP.

---

### ZeRO Stages
**Problem:** Adam stores: params (1×) + gradients (1×) + momentum (1×) + variance (1×) = 4× memory

**ZeRO-1:** Partition optimizer states → each GPU stores 1/N of Adam states
**ZeRO-2:** + partition gradients → further 2× reduction
**ZeRO-3:** + partition parameters → each GPU has 1/N of full model

**Trade-off:** ZeRO-3 = max memory efficiency but 50% more communication (must gather params before every forward/backward).

**When to use:** ZeRO-2 for <10B params, ZeRO-3 for >70B params on limited GPUs.

---

## Context Length

### Why Context Size Matters
**Compute cost:** Attention is $O(n^2)$. Doubling context = 4× memory and compute.

**Position encoding limits:**
- Sinusoidal/learned: Hard wall at training length
- RoPE: Can extrapolate via frequency interpolation

**Real numbers:** 4K context costs ~$0.0001/token. 32K costs ~$0.0006/token (6× cost for 8× length due to attention).

---

### RoPE Extrapolation (NTK-Aware, YaRN)
**Problem:** RoPE rotates by $\theta \cdot m$ where $m$ is position. At 2× training length, frequencies misalign.

**NTK-aware scaling:** $\theta' = \theta \cdot s^{d/(d-2)}$ where $s = L_{\text{new}}/L_{\text{train}}$
- Stretches wavelength to match new context

**YaRN (improved):** Different scaling per dimension—low frequencies scale more (capture long-range), high frequencies less (keep local precision).

**Result:** LLaMA-2 4K → 16K with <1% quality loss after brief fine-tuning.

---

## Loss & Training

### Perplexity vs Cross-Entropy
**Perplexity:** $\text{PPL} = \exp(\text{loss})$

**Intuition:** PPL=20 means "model is as confused as uniform random choice from 20 tokens."

**Useful conversions:**
- Loss 4.0 → PPL 54.6 (barely better than random on 50K vocab)
- Loss 2.3 → PPL 10 (reasonable)
- Loss 1.5 → PPL 4.5 (strong)

**Why use it?** Perplexity scales linearly with "confusion," loss doesn't.

---

### Z-Loss (PaLM Stability)
**Problem:** Logits can explode to ±100, causing NaN in $\exp(100)$.

**Z-loss:** $L_z = \left(\log \sum_i e^{z_i}\right)^2$
- Penalizes large log-sum-exp (logit magnitude)

**Typical weight:** $10^{-4} \times L_z$ added to cross-entropy.

**Effect:** Keeps logits centered near zero, prevents softmax saturation.

---

### MoE Load Balancing Loss
**Problem:** All tokens route to expert #3, others unused (collapse).

**Loss:** $L_{\text{aux}} = \alpha \cdot N \sum_{i=1}^{N} f_i \cdot P_i$
- $f_i$ = fraction of tokens sent to expert $i$
- $P_i$ = average gating score for expert $i$

**Intuition:** Penalize when routing frequency × gating probability is high (expert overused).

**Typical:** $\alpha = 0.01$. Too high → forced uniform routing hurts quality.

---

## Embeddings

### Tied Input/Output Embeddings
**Standard:** Input embeddings $W_e \in \mathbb{R}^{V \times d}$, output projection $W_o \in \mathbb{R}^{d \times V}$

**Tied:** Set $W_o = W_e^T$ (share weights)

**Benefits:**
- 50% fewer params (matters when V=50K)
- Semantic consistency: "cat" embedding ≈ "cat" output direction
- Better sample efficiency

**Used in:** GPT-2, GPT-3, T5, most decoder models.

---

### Absolute vs Relative Positional Encoding
**Absolute:** Position 5 gets $\text{PE}_5$, position 10 gets $\text{PE}_{10}$

**Relative:** Attention sees *distance* $i-j$, not raw positions

**Why relative wins?** "The cat sat" vs "Yesterday, the cat sat"—relative distance from "sat" to "cat" unchanged.

**Methods:**
- **RoPE:** Rotate Q/K by position-dependent angle
- **ALiBi:** Add $-(i-j)$ bias to attention scores
- **T5:** Learned bias buckets for distances

---

## Decoding Strategies

### Greedy vs Beam vs Nucleus
**Greedy:** Pick $\arg\max$ each step
- Fast, deterministic, repetitive

**Beam search (k=5):** Keep top-5 sequences at each step, expand all
- Better quality for translation
- 5× slower, still somewhat repetitive

**Nucleus (top-p=0.9):** Sample from smallest set with cumulative prob ≥ 0.9
- Adaptive: sometimes 3 tokens, sometimes 50
- Best for creative text

**Rule of thumb:** Greedy for code, beam for structured tasks, nucleus for chat.

---

### Temperature Sampling
**Formula:** $P(t) = \frac{\exp(z_t/T)}{\sum_i \exp(z_i/T)}$

**Effect:**
- $T=0.01$: Almost greedy (argmax)
- $T=1$: Raw model distribution
- $T=2$: Very random, creative but risky

**Typical values:** 0.7 for chat, 0.1 for factual QA, 1.2 for creative writing.

---

### Repetition Penalty
**Problem:** "The the the the..." loops in greedy decoding

**Fix:** $P'(t) = \frac{P(t)}{r^{\text{count}(t)}}$ where $r=1.2$
- Seen once: divide by 1.2
- Seen twice: divide by 1.44

**Warning:** $r>1.5$ makes output nonsensical (avoids all common words like "the").

---

## Quantization

### Symmetric vs Asymmetric
**Symmetric INT8:** Map $[-\alpha, \alpha] \to [-127, 127]$
- $q = \text{round}(x/s)$ where $s = \alpha/127$
- Zero point = 0 (faster, no offset addition)

**Asymmetric UINT8:** Map $[\text{min}, \text{max}] \to [0, 255]$
- $q = \text{round}(x/s) + z$
- Handles skewed distributions better

**In practice:** Symmetric for weights (faster), asymmetric for activations (better accuracy).

---

### Per-Tensor vs Per-Channel
**Per-tensor:** Single scale $s$ for entire $4096 \times 4096$ matrix
- Some channels under-utilize INT8 range

**Per-channel:** Scale $s_i$ per output channel (row)
- Reduces quantization error by 2-3×
- Minimal compute overhead (vectorized multiply)

**Always use per-channel** for weights in LLMs.

---

## Attention Optimizations

### PagedAttention (vLLM)
**Problem:** KV cache is fragmented. Seq A uses 47 tokens, Seq B uses 203 → memory holes.

**Solution:** Store KV in fixed-size pages (e.g., 16 tokens/page). Non-contiguous.

**Benefits:**
- 2-4× throughput via tighter packing
- Enables prefix sharing (same system prompt reuses KV pages)

**Analogy:** Like OS virtual memory—logical addresses map to physical blocks.

---

### Multi-Query Attention (MQA)
**Standard MHA:** Each head has separate K, V matrices

**MQA:** All heads share 1 K, 1 V (still separate Q per head)

**Trade-off:**
- 10-20× faster KV cache loading (critical for long sequences)
- ~2% quality loss vs MHA

**Who uses it:** PaLM, Falcon, StarCoder.

**Grouped-Query Attention (GQA):** Middle ground—8 heads share 2 KV pairs. Used in Llama-2 70B.

# Set-1

## QnA -1

## 🟢 P0 – Core ML + PyTorch + Fundamentals



### How do you decide whether a model is overfitting without looking at training loss?

**Key signals:**
- **Train-val gap:** 95% train accuracy vs 70% val accuracy indicates overfitting
- **Learning curve divergence:** Validation metric plateaus or degrades while training improves
- **High variance across seeds:** Different random initializations give wildly different validation scores
- **Simple example failures:** Model fails on easy held-out cases it should generalize to
- **Production degradation:** Performance drops on recent data vs validation set (temporal overfitting)

**Quick check:** Plot train vs val curves - if they diverge after epoch N, that's your overfitting point.

***

### When does accuracy become a misleading metric? Give 2 real examples.

**Example 1: Imbalanced classes**
- Fraud detection with 99.5% legitimate transactions
- Model predicting "not fraud" always = 99.5% accuracy but 0% fraud recall
- **Use instead:** Precision/Recall, F1, ROC-AUC, PR-AUC

**Example 2: Varying misclassification costs**
- Cancer diagnosis where false negative (missing cancer) >> false positive (false alarm)
- 90% accuracy can hide 80% of cancer cases if cancer is 1% of dataset
- **Use instead:** Class-specific recall, cost-weighted metrics, confusion matrix analysis

***

### Explain bias–variance tradeoff using a real product scenario.

**Scenario:** Product recommendation system

**High bias (underfitting):**
- Simple rule: "recommend most popular items"
- Low train AND test performance
- Doesn't capture user-item preference patterns

**High variance (overfitting):**
- Deep model memorizing exact user-item clicks
- Perfect on training users, terrible on new users
- Recommends obscure items clicked once by accident

**Sweet spot:**
- Captures genuine preference patterns without memorizing noise
- Use: Regularization (L2, dropout), early stopping, cross-validation
- Monitor: Train performance (bias check) + train-val gap (variance check)

***

### What are common sources of data leakage in industry datasets?

**Temporal leakage:** Using `churn_date` to predict churn (feature created after target event)

**Target leakage:** `days_since_purchase` when predicting purchase likelihood (derived from target)

**Train-test contamination:** Same user/entity in both sets without proper time-based splitting

**Preprocessing leakage:**
- `fit_transform()` on full dataset, then split
- **Correct:** `fit()` on train, `transform()` on val/test

**Proxy variables:** `cancellation_reason` to predict cancellation (only exists post-event)

**Data duplication:** Near-duplicates across train/test (common in web scraping)

***

### Why does increasing model capacity sometimes hurt generalization?

**Insufficient data constraint:** Large parameter space → fits noise instead of signal

**Optimization difficulty:** Harder to find good minima in high-dimensional spaces (many local minima, saddle points)

**Spurious correlations:** More capacity = more ability to latch onto non-transferable patterns

**Double descent:** In interpolation regime (model fits training data perfectly), increased capacity hurts before it helps again

**Loss of implicit regularization:** Very large models escape the "beneficial overfitting" zone where SGD naturally generalizes

***

### How do you choose a validation strategy when data is time-dependent?

**Time-based splits:** Never random - prevents future leakage

**Walk-forward validation:**
- Train [t₀, t₁] → validate [t₁, t₂]
- Train [t₀, t₂] → validate [t₂, t₃]

**Gap period:** Leave gap between train/val to simulate production delay (e.g., model trained Monday, deployed Wednesday)

**Seasonality:** Validate on similar time periods (same weekday, holidays)

**Multiple windows:** Check consistency across different time periods to detect temporal instability

***

## Optimization & Training Stability

### Why does a model's loss sometimes plateau early even though gradients are non-zero?

**Learning rate too small:** Gradients point correctly but step size tiny → extremely slow convergence

**Saddle points:** Gradients small (not zero) but pointing in flat directions → no progress

**Poor conditioning:** Some parameters need 10× larger updates than others, but using same LR for all

**Gradient noise:** Batch gradients fluctuating too much (high variance) → steps cancel out

**Representational limit:** Architecture can't express the solution (e.g., linear model on XOR problem)

**Mild vanishing gradients:** Early layer gradients exist but too small (~1e-8) to update effectively

***

### What are symptoms of a bad learning rate? How do you diagnose it?

**Too high:**
- Loss → NaN or inf within first epoch
- Loss oscillates wildly without downward trend
- Gradient norms spike (>1000×)

**Too low:**
- Loss barely moves after 10 epochs
- Train and val loss nearly identical (not learning)

**Diagnosis:**
- **LR range test:** Train with exponentially increasing LR (1e-6 → 1), plot loss vs LR
- **Gradient norm check:** Should be stable ~0.1-10, not exploding/vanishing
- **Parameter update ratio:** Updates should be 0.001-0.01× parameter magnitude
- **Parallel search:** Try [1e-5, 1e-4, 1e-3] on small subset quickly

***

### Why does batch size affect generalization?

**Small batches (better generalization):**
- High gradient noise → implicit regularization
- Explore more of loss landscape → escape sharp minima
- Sharp minima = high curvature = sensitive to perturbations → poor test performance

**Large batches (worse generalization):**
- Converge to sharp minima (less noise to escape)
- Less stochasticity = less exploration

**Math:** Generalization gap ∝ (LR × batch_size) / training_time

**Mitigation for large batches:** Higher LR, longer warmup, label smoothing, larger model

***

### When would you prefer Adam over SGD — and vice versa?

**Prefer Adam:**
- Prototyping / quick iteration (works out-of-box)
- Sparse gradients (NLP embeddings) - adaptive LR helps infrequent updates
- Different parameter scales handled automatically
- Limited hyperparameter search time

**Prefer SGD + momentum:**
- Final performance critical (often generalizes 1-2% better)
- Well-studied architecture (ResNets, Vision Transformers)
- Long training budget (can afford LR schedule tuning)
- Memory constrained (Adam stores 2× extra buffers: first moment + second moment)

**Why SGD generalizes better:** Adam's adaptive LR can lead to sharper minima in some settings

***

### What happens if you train with very small batches?

**High gradient variance:** Each update noisy → loss curve jagged

**Slower wall-clock time:** Can't utilize GPU parallelism (GPU underutilized at batch=1)

**Batch statistics unstable:** BatchNorm with batch=1 or 2 → running mean/var unreliable

**Convergence issues:** Batch=1 may bounce around randomly, never converge

**Can help generalization:** IF you can afford training time (extreme noise = regularization)

**Practical floor:** Usually batch ≥ 16-32 for stable training

***

### Why does normalization (BatchNorm/LayerNorm) stabilize training?

**Reduces internal covariate shift:** Layer inputs don't change distribution wildly during training

**Allows higher LR:** Prevents exploding/vanishing activations → gradients stay in healthy range

**Smooths loss landscape:** Creates more predictable gradients (less curvature)

**Implicit regularization:** BatchNorm adds noise from batch statistics → acts as regularizer

**Gradient flow:** Keeps activations in linear region of activation functions (prevents saturation)

**Reduces initialization sensitivity:** Poor init partially corrected by normalization

***

### How would you identify & tackle Vanishing Gradient problem?

**Identification:**
- Gradient norms in early layers → 0 (check `layer.weight.grad.norm()`)
- Early layers don't update (weights frozen)
- Loss plateaus but later layers still learning

**Solutions:**
- **Initialization:** He (ReLU), Xavier/Glorot (tanh/sigmoid) to maintain variance
- **Activation:** ReLU/GELU/SwiGLU instead of sigmoid/tanh (no saturation)
- **Architecture:** Residual connections (gradients flow through skip path)
- **Normalization:** BatchNorm/LayerNorm stabilizes gradient magnitudes
- **Gated RNNs:** LSTM/GRU control information flow better than vanilla RNN

***

### How would you identify & tackle Exploding Gradient problem?

**Identification:**
- Gradient norms spike (>100)
- Loss → NaN or oscillates wildly
- Weights become huge suddenly

**Solutions:**
- **Gradient clipping:** `torch.nn.utils.clip_grad_norm_(params, max_norm=1.0)` rescales to threshold
- **Weight regularization:** L2 penalty discourages large weights
- **Lower LR:** Prevents large updates from destabilizing
- **Batch Normalization:** Reduces dependency on initialization, manages gradient magnitudes

***

## PyTorch / DL Internals

### What exactly does autograd track, and when does it fail?

**Tracks:**
- Computational graph of operations on `requires_grad=True` tensors
- Backward function for each operation
- Intermediate activations needed for chain rule

**Fails on:**
- **In-place ops:** `x += 1` corrupts graph (use `x = x + 1`)
- **Detached tensors:** `.detach()` or `.data` breaks gradient flow
- **Non-PyTorch ops:** NumPy operations, Python conditionals
- **Non-differentiable ops:** `torch.argmax()`, integer indexing
- **Graph cleared:** After `.backward()`, need `retain_graph=True` for second backward

***

### Why can a PyTorch model silently stop learning?

**Forgot `optimizer.zero_grad()`:** Gradients accumulate → nonsensical updates

**Detached tensors:** `loss = loss.detach()` breaks gradient flow

**Wrong loss function:** MSE for classification, or using probabilities when expecting logits

**Dead neurons:** All ReLU outputs ≤ 0 → zero gradients forever

**Forgot `optimizer.step()`:** Gradients computed but weights not updated

**LR decayed to ~0:** Scheduler reduced LR to 1e-10

**`requires_grad=False`:** Parameters or inputs don't track gradients

***

### What happens if you forget to call `model.eval()` during validation?

**Dropout stays active:** Randomly zeros neurons → pessimistic metrics

**BatchNorm uses batch statistics:** Instead of running mean/var → noisy, unreliable

**Consequences:**
- Validation metrics worse than actual capability
- Can't reproduce results (metrics change with batch composition)
- Misleading hyperparameter decisions

**Critical production bug:** Many teams shipped models without catching this

***



# QnA - 2

## 🟡 P1 – LLMs, Transformers & GenAI Systems

### Why does self-attention scale poorly with sequence length?

**Complexity:** \(O(n^2)\) in both memory and compute where \(n\) = sequence length

**Math:** For each of \(n\) tokens, compute attention to all \(n\) tokens → \(n \times n\) matrix

**Memory:** Must store \([batch, heads, seq, seq]\) attention matrix
- 1000 tokens → 1M pairwise scores
- 4096 tokens → 16M scores per head

**Compute:** Matrix multiplication \(QK^T\) is \(O(n^2 d)\), softmax over \(n^2\) elements

**Workarounds:**
- Sparse attention (Longformer: local + global patterns)
- Linear attention (approximations)
- Flash Attention (tiling to avoid materializing full matrix)
- Sliding window (Mistral: 4096-token window)

***

### Encoder vs decoder-only models — when do you use each?

**Encoder-only (BERT):**
- **Attention:** Bidirectional (sees full context)
- **Use cases:** Classification, NER, embeddings, Q&A extraction (when you have full input)
- **Can't:** Generate text autoregressively

**Decoder-only (GPT):**
- **Attention:** Causal (only left context)
- **Use cases:** Text generation, chat, completion, code generation
- **Why dominant:** Simpler architecture, scales better, unified pretraining = generation

**Encoder-decoder (T5):**
- **Structure:** Encoder (bidirectional) + decoder (causal)
- **Use cases:** Translation, summarization (different input/output)
- **Less common now:** Decoder-only can do these via prompting

***

### Why do LLMs hallucinate even when prompted carefully?

**Training objective:** Next-token prediction ≠ factual accuracy (optimizes plausibility, not truth)

**No grounding:** Pattern matching from training data, no access to external facts

**Overconfidence:** High probability doesn't mean factually correct (just statistically likely continuation)

**Conflicting training data:** Seen different "facts" about same topic

**Can't say "I don't know":** Always generates something (sampling from distribution)

**Context limitations:** Long conversations lose track of what was stated vs inferred

**Prompt misinterpretation:** Model's "understanding" differs from user intent

***

### What changes between pretraining and fine-tuning?

**Pretraining:**
- **Objective:** Next-token prediction on massive unlabeled corpus
- **Data:** Trillions of tokens (web crawl, books, code)
- **Goal:** Learn language patterns, world knowledge, reasoning
- **Cost:** $millions, months of training
- **LR:** 1e-4 to 6e-4 typical

**Fine-tuning:**
- **Objective:** Task-specific (instruction following, Q&A, summarization)
- **Data:** Thousands to millions of curated examples
- **Goal:** Align to specific behavior/format
- **Cost:** Much cheaper (single GPU, hours/days)
- **LR:** 1e-5 to 1e-6 (much smaller to avoid catastrophic forgetting)

***

### Why does instruction tuning improve usability?

**Alignment:** Model learns to follow user intent, not just predict text

**Format consistency:** Responds in expected structure (markdown, code blocks, numbered lists)

**Reduced prompt engineering:** Works with natural prompts instead of careful formatting

**Safety:** Can refuse harmful requests, stay on topic

**Zero-shot improvement:** Better at new tasks without examples

**Conversational behavior:** Learns helpful assistant patterns (clarifying questions, admitting uncertainty)

***

### What is the role of positional encodings?

**Core problem:** Attention is permutation-invariant → "cat sat mat" = "mat sat cat" without positions

**Purpose:** Inject token position information so model knows word order

**Types:**

**Absolute:**
- Sinusoidal: $(PE_{(pos, 2i)} = \sin(pos/10000^{2i/d}))$
- Learned embeddings (GPT-2)
- **Issue:** Can't extrapolate beyond training length

**Relative (modern):**
- **RoPE:** Rotates Q/K vectors by position-dependent angle (LLaMA, GPT-NeoX)
- **ALiBi:** Adds bias $(-m|i-j|)$ to attention scores (BLOOM, MPT)
- **Benefit:** Generalizes to longer sequences

**Why it matters:** Enables syntax understanding, temporal reasoning, dependency parsing

***

## Inference & Behavior

### Why is LLM inference slow compared to training per token?

**Training:** Batched - process 1000s of sequences in parallel → high GPU utilization

**Inference:** Sequential autoregressive - must generate one token at a time

**Memory-bound:** Constantly loading weights from VRAM (not compute-bound like training)

**KV-cache management:** Store/retrieve cached keys/values for all previous tokens

**Low batch size:** Often serving 1 user at a time (can't parallelize across batch like training)

**Per-token cost:** Each token requires full forward pass through all layers

**Key difference:** Training processes many tokens simultaneously; inference generates one token per step

***

### What happens during prefill vs decoding?

**Prefill (prompt processing):**
- Process all prompt tokens **in parallel**
- Compute KV cache for entire prompt at once
- Single forward pass for N prompt tokens
- Fast - fully utilizes GPU (batch matrix ops)

**Decoding (generation):**
- Generate tokens **sequentially** one at a time
- Each step: attend to all previous tokens (growing KV cache)
- N tokens = N separate forward passes
- Slow - can't parallelize (autoregressive dependency)

**Optimization focus:**
- Prefill → maximize throughput
- Decoding → minimize per-token latency

***

### Why does temperature affect factual accuracy?

**Temperature = 0 (greedy):** Always pick $(\arg\max P(token))$
- Deterministic, most factual
- Can get stuck in loops

**Low temperature (0.1-0.5):** Slightly randomized
- Reduces repetition
- Still mostly factual

**High temperature (0.8-1.5):** Flatter distribution

**Math:** $P(token) = \frac{e^{logit/T}}{\sum_i e^{logit_i/T}}$

**Effect of high T:**
- Samples from lower-probability (less reliable) tokens
- More creative but less accurate
- Can drift into hallucinations

**Use cases:**
- Factual Q&A → T=0.1-0.3
- Creative writing → T=0.7-1.0
- Code generation → T=0.1-0.2

***

### When does increasing context length degrade performance?

**Lost in the middle:** Model struggles to use information from middle of long contexts (proven empirically)

**Attention dilution:** Attention spread thin across many tokens → weaker signal per token

**Position extrapolation:** Trained on 2K context, using 8K may break positional encodings

**Noise accumulation:** More irrelevant text to filter through

**Practical observation:** Performance often peaks at 4K-8K tokens, then degrades beyond training length

**When it helps:** Specific info retrieval, long documents where answer is definitely in context

***

### Why do LLMs struggle with counting or exact arithmetic?

**Tokenization inconsistency:** `1234` → `["12", "34"]` or `["1", "234"]` (varies by tokenizer)

**No symbolic reasoning:** Pattern matching, not actual computation

**Position tracking:** Counting requires precise positional awareness across many tokens

**Training bias:** Seen more small numbers than large (data distribution skew)

**Approximation by design:** Learned to approximate, not compute exactly

**No scratchpad:** Can't do multi-step calculation systematically (unless using chain-of-thought)

**Workarounds:** Chain-of-thought prompting, tool use (calculator APIs), code generation

***

## Prompting vs Finetuning

### When is prompt engineering insufficient?

**Consistent format required:** Need exact JSON structure every time (prompts can be brittle)

**Domain-specific knowledge:** Technical jargon not in pretraining data

**Latency constraints:** Long prompts (500+ tokens) slow inference

**Cost at scale:** Prompts expensive when serving millions of requests (pay per token)

**Reliability:** Can't afford prompt brittleness in production

**Complex multi-step reasoning:** Need learned shortcuts, not verbose prompting

**Style/tone:** Consistent brand voice hard to maintain via prompting alone

***

### How do you decide between: Prompting, Fine-tuning, LoRA/adapters?

**Prompting:**
- **When:** Quick prototyping, low volume (<1K requests/day), task known to base model
- **Pros:** No training, flexible, fast iteration
- **Cons:** Token cost, brittleness, less control

**LoRA (sweet spot for most):**
- **When:** Need customization, limited budget/data
- **Pros:** Train 0.1-1% of parameters → 100× faster/cheaper than full fine-tuning
- **Pros:** Multiple adapters on one base model, lower forgetting risk
- **Cons:** Slightly lower performance than full fine-tuning

**Full Fine-tuning:**
- **When:** High volume, maximum performance needed, have compute budget
- **Pros:** Best performance, fully customized
- **Cons:** Expensive, risk catastrophic forgetting, separate model to maintain

**Decision tree:**
1. Try prompting first (cheapest validation)
2. If prompt >300 tokens or unreliable → LoRA
3. If performance critical and have budget → full fine-tune

***

### Why can fine-tuning reduce generalization?

**Catastrophic forgetting:** Overwrites general knowledge with task-specific patterns

**Narrow distribution:** Training on medical Q&A → worse at coding, creative writing

**Format overfitting:** Learns to match training format, not understand deeply

**Example:** Fine-tune on customer support → loses capability on math, translation, code

**Mitigation:**
- Use LoRA (freezes 99% of model)
- Mix general data (20-30%) with task data during fine-tuning
- Lower LR (1e-6 instead of 1e-4)
- Early stopping

***

### What are risks of overfitting during instruction tuning?

**Memorizing examples:** Repeats exact training responses word-for-word

**Format lock-in:** Only works with training-style prompts, fails on variations

**Reward hacking:** Learns to game evaluation metrics instead of actual quality

**Sycophancy:** Agrees with user even when wrong (if trained on always-agreeing data)

**Loss of breadth:** Forgets how to do tasks not in fine-tuning data

**Signs:**
- Perfect train metrics, poor eval
- Responses sound "canned"
- Fails on rephrased questions

***

## Evaluation & Reliability

### Why is BLEU/ROUGE insufficient for LLM evaluation?

**Surface-level matching:** Only measures n-gram overlap, ignores semantics

**Example:** "The car is red" vs "The vehicle is crimson" = low BLEU but same meaning

**Multiple correct answers:** "Yes, definitely" vs "Absolutely" = low score but both correct

**Can't measure:**
- Factuality (fluent nonsense scores high)
- Reasoning quality
- Helpfulness
- Safety

**Better alternatives:**
- LLM-as-judge (GPT-4 scoring responses)
- Human evaluation
- Task-specific metrics (exact match for extraction, accuracy for Q&A)
- Embedding similarity (semantic, not n-gram)

***

### How do you detect hallucinations systematically?

**Factual consistency:**
- Cross-reference with knowledge base
- Check citations exist and support claims
- Compare with search results

**Self-consistency:**
- Ask same question 3 different ways
- Check if answers align

**Confidence scoring:**
- Monitor token probabilities (low confidence often = hallucination)
- Aggregate uncertainty across tokens

**Adversarial probing:**
- Ask about non-existent entities (e.g., "Tell me about the 2025 Mars Olympics")
- Nonsense questions to test if model refuses

**Retrieval validation (RAG):**
- Check if answer grounded in retrieved docs
- Attribution matching

**Automated tools:**
- Fact-checking APIs
- NER verification
- LLM-based hallucination detectors

***

### What does "LLM drift" mean?

**Definition:** Model behavior changes over time without explicit updates

**Causes:**
- Provider updates model weights (fine-tuning, RLHF)
- Input distribution shifts
- Context/prompt caching issues
- API version changes

**Symptoms:**
- Previously working prompts fail
- Output format changes
- Different responses to identical inputs
- Performance degradation on benchmarks

**Mitigation:**
- Pin specific model versions (`gpt-4-0613` not `gpt-4`)
- Regular evaluation on held-out test set
- Monitor output distribution (log predictions)
- A/B test before deploying new versions

***

### How do you evaluate changes in prompts safely?

**Held-out test set:** Never tune prompts on eval data (contamination)

**Diverse examples:** Cover edge cases, different input types, failure modes

**Regression testing:** Ensure new prompt doesn't break existing cases

**A/B testing:** Run both prompts on 5% → 20% → 50% → 100% traffic

**Multiple evaluators:** Human raters + automated metrics (avoid single point of failure)

**Version control:** Git-track all prompts with performance metadata

**Key metrics:**
- Task accuracy/F1
- Human preference score
- Latency (longer prompts = slower)
- Failure rate (% of malformed outputs)

***

### Why do offline metrics often disagree with user feedback?

**Proxy vs real goal:** ```Optimizing BLEU ≠ user satisfaction```

**Missing context:** Users have intent/background not in test set

**Subjective quality:** "Helpfulness" hard to metric (tone, conciseness preferences vary)

**User expectations evolve:** What's "good" changes over time, varies by user segment

**Latency matters:** Slow but perfect < fast but good enough (offline metrics ignore speed)

**Business metrics differ:** User retention/engagement ≠ model accuracy

**Solution:** Combine offline metrics + online A/B tests + user feedback loops

***


## QnA -3
## 🟠 P2 – Applied NLP / DL Depth




### Why do embeddings capture semantic similarity?

**Distributional hypothesis:** "Words in similar contexts have similar meanings"

**Training objective:** Predict context (Word2Vec) or masked tokens (BERT)
- Minimize loss → similar usage patterns → similar vectors

**Geometry emerges:** Distance in embedding space = semantic distance

**Math:** Words that co-occur frequently get gradients pulling them closer

**Example:** "king" and "queen" appear in similar contexts (royalty, ruler) → close embeddings

**Compositional:** Vector arithmetic works: \(\vec{king} - \vec{man} + \vec{woman} \approx \vec{queen}\)

***

### When do cosine similarity and dot product behave differently?

**Cosine similarity:**
\[\text{cos}(\theta) = \frac{A \cdot B}{||A|| \cdot ||B||}\]
- Measures angle, ignores magnitude
- Range: [-1, 1]
- **Use when:** Vector magnitude meaningless (text embeddings)

**Dot product:**
\[A \cdot B = ||A|| \cdot ||B|| \cdot \text{cos}(\theta)\]
- Measures angle AND magnitude
- Range: unbounded
- **Use when:** Magnitude matters (attention scores in transformers)

**Key difference:**
- Normalized embeddings: cosine = dot product
- Unnormalized: dot product prefers longer vectors

**Practice:** Most retrieval systems normalize embeddings, then use dot product (faster than cosine)

***

### Why does embedding quality degrade for long documents?

**Averaging problem:** Mean pooling of 1000 tokens dilutes specific information

**Fixed dimensionality bottleneck:** 768-dim vector must compress thousands of tokens

**Attention dilution:** Model can't focus equally on all parts

**Context limits:** Exceeds training context length (e.g., BERT trained on 512 tokens)

**Information loss:** Compressing too much into fixed size

**Solutions:**
- Chunk documents, embed separately
- Weighted pooling (attention-based aggregation)
- Hierarchical embeddings (chunk → document)
- Long-context models (Longformer)

***

### What causes embedding collapse?

**Definition:** All embeddings converge to same/very similar vectors

**Causes:**
- Poor loss function (doesn't enforce separation)
- Insufficient negative sampling (no contrast)
- Too strong normalization pushing all to unit sphere center
- Model bottleneck too tight (information loss)
- Vanishing gradients

**Symptoms:**
- All cosine similarities near 1.0
- Retrieval returns random results
- Loss decreases but embeddings don't separate

**Prevention:**
- Contrastive learning with hard negatives
- Monitor embedding variance during training (should be >0.1)
- Use triplet/InfoNCE loss (enforces separation)

***

### Why does tokenization matter for performance?

**Vocabulary efficiency:** BPE/WordPiece balance vocab size vs coverage

**Sequence length:** Poor tokenization → longer sequences → slower, more memory

**Semantic preservation:** "unhappy" as 1 token vs ["un", "happy"] affects meaning capture

**OOV handling:** Character-level handles any text but too long; word-level fails on unseen words

**Cross-lingual:** Good tokenization shares subwords across languages

**Numbers/code:** Different tokenizers split differently
- Good: `"ChatGPT"` → `["Chat", "GPT"]`
- Bad: `"ChatGPT"` → `["Chat", "G", "PT"]`

**Impact:** Can cause 10-20% performance difference on domain-specific tasks

***

### How can tokenization introduce bias?

**Language bias:**
- English word = 1 token
- Same Chinese word = 3-4 tokens
- **Impact:** Chinese users pay 3× more, slower inference, less context

**Name bias:**
- "John Smith" = 2 tokens
- "Sundarajan Pichai" = 6 tokens
- **Impact:** Non-Western names consume more context window

**Domain bias:**
- "FinTech" → `["Fin", "Tech"]` vs `["FinTech"]`
- Technical terms split badly if not in training data

**Case sensitivity:**
- "Hello" and "hello" as separate tokens (wasteful)

**Consequences:** Different user groups experience different model quality, cost, latency

***

### Why does truncation hurt certain NLP tasks more than others?

**Hurts more:**
- **Long-form Q&A:** Answer often at end of document (truncating = losing answer)
- **Summarization:** Need full document context
- **Legal/medical:** Critical info anywhere, can't afford to lose
- **Multi-turn dialogue:** Early conversation context needed for coherence

**Hurts less:**
- **Classification:** Often first sentences sufficient (sentiment, topic)
- **NER:** Local context usually enough
- **Short Q&A:** Context already fits

**Solutions:**
- Sliding window with overlap
- Chunk and aggregate predictions
- Hierarchical processing (summarize chunks, then aggregate)

***



## QnA -4
## 🟠P3 - Fine-Tuning,Loss Functions & Training

### Fine-Tuning & Data

### Why does fine-tuning sometimes reduce zero-shot performance?

**Catastrophic forgetting:** Weights updated to specialize → lose general capabilities

**Distribution shift:** Training on narrow domain shifts learned distribution away from general knowledge

**Format conditioning:** Model expects fine-tuning format, fails on raw/diverse prompts

**Task interference:** New task patterns overwrite old knowledge in shared parameter space

**Example:** Fine-tune on customer support → worse at creative writing, math, coding, translation

**Mitigation:**
- **LoRA:** Freezes 99% of parameters, only trains adapters
- Mix 20-30% general data during fine-tuning
- Lower LR (1e-6 vs 1e-4)
- Fewer epochs (1-3 instead of 10)

***

### What data issues cause catastrophic forgetting?

**Too specialized:** Only task-specific examples, no general data to anchor knowledge

**Too few examples:** Overfits quickly to narrow distribution (<100 examples dangerous)

**High learning rate:** Aggressively overwrites pretrained weights (use 10-100× smaller LR than pretraining)

**Too many epochs:** First epoch helps, epoch 10 causes forgetting

**Imbalanced data:** Model forgets underrepresented capabilities

**Format mismatch:** Very different format from pretraining (e.g., only JSON outputs)

**No regularization:** Full fine-tuning without constraints

***

### How much data is "enough" for fine-tuning?

**Rule of thumb:**

**Instruction tuning (general):** 1K-10K diverse examples

**Domain adaptation:** 10K-100K examples

**Specific narrow task:** 100-1K examples

**Quality > quantity:**
- 1K high-quality, diverse examples > 10K repetitive ones
- Need coverage of edge cases, variations, failure modes

**Signs of insufficient data:**
- Memorizing training examples verbatim
- High variance between runs (different seeds = different models)
- Train accuracy 95%, val accuracy 60%

**Modern reality with LoRA:**
- Can fine-tune effectively with 500-1K examples
- Less data needed than full fine-tuning (less overfitting risk)

***

### Why does class imbalance affect neural models differently than tree models?

**Tree models (Random Forest, XGBoost):**
- Each split can find patterns in minority class
- Ensemble averaging helps
- Naturally handle imbalance via feature splits

**Neural networks:**
- **Gradient-based learning biased toward majority class**
- Loss dominated by majority: \(L = \frac{1}{N}\sum L_i\) where 99% are majority
- Updates mostly help majority (minority barely affects gradients)
- Can collapse to always predicting majority (99% accuracy on 99:1 imbalance!)

**Solutions for NNs:**
- **Class weights:** \(L = \sum w_i L_i\) where \(w_{minority} = 10 \times w_{majority}\)
- Focal loss: \(FL = -(1-p_t)^\gamma \log(p_t)\) (focuses on hard examples)
- Oversample minority or undersample majority
- Two-stage training (balance first, then full data)

***

## Loss Functions & Training

### Why does cross-entropy work well for language modeling?

**Probabilistic interpretation:** Naturally models \(P(\text{next token} | \text{context})\)

**Penalizes confidence on wrong predictions:** Not just binary right/wrong
- Predicting "cat" with 99% when answer is "dog" = huge loss
- Predicting "cat" with 51% when answer is "dog" = smaller loss

**Formula:** \(L = -\sum_{i} y_i \log(\hat{y}_i)\) where \(y\) is one-hot, \(\hat{y}\) is predicted distribution

**Smooth gradients:** Provides learning signal across all probabilities (not just argmax)

**Information-theoretic:** Minimizes KL divergence between predicted and true distributions

**Well-calibrated:** Output probabilities interpretable as confidence

**Natural pairing with softmax:**
\[\frac{\partial L}{\partial z_i} = \hat{y}_i - y_i\]
Clean gradient (no vanishing/exploding)

***

### When would you modify the loss instead of the model?

**Class imbalance:** Weighted cross-entropy simpler than changing architecture

**Hard negative mining:** Focal loss focuses on difficult examples without architectural change

**Multi-task learning:** Combined loss \(L = \alpha L_1 + \beta L_2\) easier than multi-head architecture

**Regularization:** Add L1/L2 penalty: \(L_{total} = L_{task} + \lambda ||W||^2\)

**Robustness:** Label smoothing for noisy labels

**Calibration:** Temperature scaling in loss for better probability estimates

**Curriculum learning:** Gradually increase loss weight on hard examples

**Advantage:** Easier to implement, tune, debug than architectural changes

***

### Why does label noise hurt deep models more?

**High capacity:** Neural networks can memorize noise (10M parameters can fit random labels)

**Confidence:** Become overconfident on mislabeled examples (softmax → 0.99 on wrong label)

**Gradient propagation:** Wrong labels send wrong signals throughout entire network

**Many epochs:** More exposure to noisy labels → more memorization

**Tree models more robust:**
- Local decisions less affected by single mislabel
- Ensemble averages out noise
- Less prone to memorizing individual examples

**Solutions:**
- Label smoothing: \(y' = (1-\epsilon)y + \epsilon/K\) where \(\epsilon=0.1\)
- Confident learning (identify and remove/reweight noisy labels)
- MixUp augmentation
- Early stopping (before memorization phase)

***



## QnA -5
## 🔵 P4 – RAG, Optimization & Practical System Design


### Why does naive top-k retrieval often fail?

**Semantic vs keyword mismatch:** Query "How to fix error?" vs document "Troubleshooting guide" (different words, same meaning)

**Query too vague:** "Tell me about it" → what's "it"?

**Query too specific:** "Python 3.11.2 bug fix" retrieves narrow results, misses general Python debugging docs

**Chunk boundaries:** Answer split across chunks (chunk 1 has question, chunk 2 has answer)

**Poor chunking:**
- Too small (100 tokens) → loses context
- Too large (1000 tokens) → retrieves noise with signal

**Embedding quality:** Generic embeddings (all-MiniLM) don't capture domain jargon

**No query understanding:** Treats "best restaurants" same as "worst restaurants" (negation lost)

**Recency bias:** Vector DB might not have latest docs

***

### How does chunk size affect retrieval quality?

**Too small (100-200 tokens):**
- **Pros:** Precise retrieval, more granular
- **Cons:**
  - Loses context (answer spans multiple chunks)
  - Need to retrieve 10+ chunks to get full context
  - Higher noise (many irrelevant chunks)

**Too large (1000+ tokens):**
- **Pros:** Better context preservation
- **Cons:**
  - Less precise (retrieves irrelevant info with relevant)
  - Fewer total chunks → worse recall
  - Embeddings less focused (averaging over too much)

**Sweet spot:** 300-500 tokens with 50-100 token overlap

**Advanced:**
- Hierarchical chunking (summaries + detail chunks)
- Semantic chunking (split by topic change, not fixed size)
- Query-dependent chunking

***

### Why does retrieval sometimes reduce answer quality?

**Retrieves wrong information:** Misleads model worse than no retrieval

**Contradicts model knowledge:** Retrieved text says "Paris is in Germany" → creates confusion

**Noise injection:** 5 retrieved chunks, only 1 relevant → other 4 distract

**Context stuffing:** Too many chunks exceed context limit → information lost or truncated

**Hallucination anchoring:** Model forced to use bad retrieval, produces worse hallucination than generating from knowledge

**Query-document mismatch:** Semantic search finds superficially similar but contextually wrong docs

**When to skip retrieval:**
- Model is confident (high token probability)
- Query is general knowledge ("What is gravity?")
- Retrieval confidence score <0.7

***

### When does reranking help most?

**Initial retrieval is noisy:** Top-10 has 7 false positives → reranking filters them

**Need precision over recall:** Better to have 3 great results than 10 mixed

**Complex matching:** Beyond simple semantic similarity
- Negation ("not Windows")
- Temporal constraints ("after 2023")
- Specific requirements ("Python 3.11+ only")

**Cost-performance tradeoff:**
- Cheap embedding search on 1000 docs → top-50
- Expensive cross-encoder reranking on top-50 → top-5

**Reranking models:**
- Cross-encoders (query + document encoded together)
- LLM-based scoring (GPT-4 rates relevance)
- Hybrid (keyword + semantic)

**When it doesn't help:**
- Initial retrieval already high quality (>0.9 relevance)
- Latency critical (reranking adds 100-300ms)

***

### How do you debug a bad RAG response?

**Check retrieval quality:**
- Are correct chunks retrieved? (manually inspect top-5)
- Similarity scores? (should be >0.7 for good retrieval)
- Try different queries (rephrase question)

**Check chunks:**
- Is answer actually in retrieved chunks? (read them!)
- Properly formatted? (no encoding issues)
- Context window not exceeded? (count tokens)

**Check query:**
- Well-formed? ("Fix bug" vs "How to fix authentication bug in FastAPI?")
- Query transformation help? (HyDE, query expansion)

**Check prompt:**
- Clear instruction? ("Use only retrieved context")
- Chunks properly injected? (verify format)
- Does model ignore retrieval? (try with/without retrieval)

**Check model:**
- Run without retrieval → does it hallucinate?
- Temperature too high? (try 0.1)
- Model too small? (7B struggles vs 70B)

**Debugging tools:**
- Log all retrieved chunks + scores
- Similarity score distribution (should vary, not all 0.8)
- Try different retrievers (BM25 vs embedding)

***

## Vector Databases & Search

### What are common causes of low recall in vector search?

**Poor embeddings:** Generic model (all-MiniLM-L6) doesn't capture domain nuances (medical, legal)

**Embedding drift:** Query embedding model ≠ document embedding model (or different versions)

**Wrong distance metric:** Using L2 when embeddings designed for cosine

**ANN index too aggressive:** HNSW with `ef_search=10` trades recall for speed (try 50-100)

**Query too short/vague:** "error" → not enough signal for good embedding

**Document preprocessing:** Missing important text, bad chunking

**Vocabulary mismatch:** Domain jargon not in embedding model training data

**Vector dimensionality:** 384-dim may lose information vs 768-dim

**Diagnosis:**
- Compare vector search vs BM25 recall
- Manually inspect missed documents
- Visualize embeddings (t-SNE/UMAP - are clusters meaningful?)
- A/B test embedding models

***

### Why does semantic similarity fail for some queries?

**Exact match needed:** "Python 3.11 bug" needs keyword, not semantic (3.10 semantically similar but wrong)

**Negation:** "not Windows" vs "Windows" have similar embeddings (negation lost)

**Numbers/dates:** "2023 report" vs "2024 report" embed similarly (temporal info lost)

**Acronyms:** "ML" vs "Machine Learning" may not be close depending on tokenization

**Multi-intent:** "Compare X and Y" needs both X and Y, semantic search may retrieve only X

**Domain-specific:** "Bullish sentiment" in finance ≠ general "positive"

**Solutions:**
- Hybrid search (BM25 + semantic)
- Query preprocessing (expand acronyms, extract entities)
- Fine-tune embeddings on domain data
- Metadata filtering (date ranges, exact fields)

***

### How do you choose between ANN algorithms (HNSW, IVF, etc.)?

**HNSW (Hierarchical Navigable Small World):**
- **Pros:** Best recall-speed tradeoff, query time fast
- **Cons:** High memory (stores full graph), slow indexing
- **Use when:** Memory abundant, query latency critical
- **Params:** `ef_construction=200, M=16` (higher = better recall, slower)

**IVF (Inverted File Index):**
- **Pros:** Lower memory, faster indexing
- **Cons:** Needs training step, recall-speed tradeoff
- **Use when:** Large datasets (>10M vectors), memory constrained
- **Params:** `nlist` (clusters), `nprobe` (clusters to search)

**Flat (exact search):**
- **Pros:** Perfect recall
- **Cons:** Slow (\(O(n)\) per query)
- **Use when:** <100K vectors, or as baseline

**Product Quantization (PQ):**
- **Pros:** Extreme compression (32× smaller)
- **Cons:** Recall loss
- **Use when:** Billions of vectors, memory severely constrained

**Decision tree:**
- <100K vectors → Flat
- 100K-10M, memory OK → HNSW
- 10M-1B → IVF or HNSW with PQ
- >1B → IVF + PQ

***

### What causes index performance to degrade over time?

**Index fragmentation:** Many incremental updates create inefficient structure (like disk fragmentation)

**Imbalanced partitions:** IVF clusters become skewed (some large, some empty)

**Memory pressure:** Cache misses increase as index grows beyond RAM

**Stale statistics:** ANN algorithms rely on distribution assumptions that drift

**No retraining:** IVF trained on initial 1M vectors, now has 10M (centroids outdated)

**Too many deletions:** Tombstone records accumulate (marked deleted but still in index)

**Solutions:**
- Rebuild index periodically (weekly/monthly)
- Retrain IVF centroids on fresh sample
- Merge segments (like Elasticsearch)
- Monitor query latency metrics

***

## Distributed Training & Optimization

### Why does mixed precision training work?

**FP16 benefits:**
- 2× less memory (activations, gradients)
- 2-3× faster on modern GPUs (Tensor Cores)
- Larger batch sizes fit

**Problem:** FP16 range too small → gradients underflow to zero

**Solution (Loss Scaling):**
1. Scale loss by large factor (e.g., 1024)
2. Backward pass: gradients scaled up → don't underflow
3. Before optimizer step: scale gradients back down
4. Update weights

**Math:**
\[
\text{grad}_{scaled} = 1024 \times \text{grad}_{actual}
\]
\[
\text{grad}_{final} = \text{grad}_{scaled} / 1024
\]

**BFloat16 (better):**
- Same exponent range as FP32 (8 bits)
- No loss scaling needed
- Slightly less precise mantissa (7 bits vs 10 in FP16)
- Used in TPUs, A100 GPUs

***

### What is gradient accumulation and when do you use it?

**Idea:** Simulate large batch by accumulating gradients over \(N\) small batches

**Implementation:**
```python
for i, batch in enumerate(loader):
    loss = model(batch) / N  # normalize by accumulation steps
    loss.backward()  # accumulate gradients
    
    if (i+1) % N == 0:
        optimizer.step()
        optimizer.zero_grad()
```

**Effect:** Effective batch = \(N \times \text{batch\_size}\)

**When to use:**
- GPU memory limited (can't fit large batch)
- Want large effective batch for stability
- Distributed training with few GPUs

**Trade-offs:**
- Slower wall-clock time (N forward/backward before update)
- Identical gradient to true large batch (mathematically equivalent)
- BatchNorm breaks (but transformers use LayerNorm, so fine)

***

### Why does learning rate warmup help?

**Problem:** At start of training, weights random → gradients large and chaotic

**Large LR at start:** Can cause:
- Exploding gradients
- Divergence
- Poor optimization trajectory (get stuck in bad local minimum)

**Warmup:** Linearly increase LR from 0 to target over first \(N\) steps

**Math:**
\[
\text{LR}(step) = \text{LR}_{max} \times \frac{step}{warmup\_steps}
\]

**Why it works:**
- Small LR initially → stable updates while model finds general direction
- Gradually increase → speeds up once stabilized

**Typical:** 2-10% of total training steps (e.g., 1000 steps warmup for 10K total)

**Especially important for:**
- Very large models (billions of parameters)
- Large batch sizes (>1024)
- Adam optimizer (adaptive LR can be unstable early)

***

### What is gradient clipping and when is it necessary?

**Definition:** Cap gradient norm to maximum value

**Implementation:**
```python
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
```

**Math:**
\[
\text{if } ||\nabla|| > \text{max\_norm}: \quad \nabla = \nabla \times \frac{\text{max\_norm}}{||\nabla||}
\]

**When necessary:**
- **RNNs/LSTMs:** Gradients multiply across time steps → explode
- **Very deep networks:** Without residual connections
- **Reinforcement learning:** Policy gradients can spike
- **Mixed precision training:** Rare but extreme outliers
- **Long sequences:** Transformers on 4K+ token sequences

**Typical values:**
- Conservative: `max_norm=0.5`
- Standard: `max_norm=1.0`
- Loose: `max_norm=5.0`

**Check if needed:** Log gradient norms - if seeing values >10 frequently, enable clipping

***

### How does learning rate scheduling improve convergence?

**Constant LR problems:**
- Too high → oscillates around minimum, never settles
- Too low → slow convergence

**Scheduling:** Reduce LR during training

**Common schedules:**

**Cosine annealing:**
$
\text{LR}(t) = \text{LR}_{min} + \frac{1}{2}(\text{LR}_{max} - \text{LR}_{min})(1 + \cos(\frac{t\pi}{T}))
$
- Smooth decrease
- Used in GPT-3, LLaMA

**Step decay:**
$
\text{LR}(epoch) = \text{LR}_{init} \times 0.1^{\lfloor epoch/30 \rfloor}
$
- Drop by 10× every 30 epochs
- Simple, effective for CNNs

**Inverse square root:**
$
\text{LR}(t) = \text{LR}_{init} / \sqrt{t}
$
- Used in original Transformer

**Why it works:**
- High LR early → fast exploration
- Low LR late → fine-tune near minimum

**With warmup:** Combine both (warmup + cosine annealing) for best results

***


# Set 2

## QnA - 1

## 🟢 P0 – Core ML, Training & Debugging





### Data & Generalization

**1. Why can adding more data reduce performance?**

**Math insight**: Model minimizes empirical risk $R_{emp}(\theta) = \frac{1}{n} \sum_{i=1}^{n} L(f_\theta(x_i), y_i)$. When new data changes the underlying distribution $P(x,y)$, optimizing on mixed distribution can hurt:
- If $P_{new}(y|x)$ conflicts with $P_{old}(y|x)$, gradients fight each other
- Rare but important examples have weight $\frac{1}{n}$ → as $n$ grows, weight → 0

**Key reasons**:
- **Distribution mismatch**: New data from different sources/labeling standards ($P_{old} \neq P_{new}$)
- **Capacity constraint**: Model has fixed VC dimension—can't capture added complexity
- **Signal dilution**: Important patterns get weight $\frac{1}{n}$ in loss, swamped by noise

**Solution**: Check KL-divergence or JS-divergence between old/new distributions. For mixed sources (Amazon reviews + Twitter), use importance weighting: $w_i = \frac{P_{target}(x_i)}{P_{train}(x_i)}$. Monitor per-source metrics separately.

---

**2. How do you detect covariate shift without labels?**

**Math**: Testing if $P_{train}(X) \neq P_{deploy}(X)$ while $P(Y|X)$ stays same

**Methods**:
- **Discriminator approach**: Train classifier $D(x): \{train, deploy\}$. If accuracy $> 0.75$ → $H_1$: distributions differ
- **Statistical tests**: Two-sample tests
  - Kolmogorov-Smirnov: $D = \sup_x |F_{train}(x) - F_{deploy}(x)|$
  - Maximum Mean Discrepancy: $||E_{P_{train}}[\phi(x)] - E_{P_{deploy}}[\phi(x)]||_H$
- **Monitor drift**: Track $H(p)$ (entropy) or $KL(p_{old} || p_{new})$ of prediction distributions over time

**Solution**: Start with RandomForest discriminator—feature importance shows which features shift. Validate with statistical tests on top shifters. Use PCA visualization for exploration.

---

**3. When prefer high-variance, low-bias models?**

**Math**: Bias-variance decomposition of MSE:
$E[(y - \hat{f}(x))^2] = Bias[\hat{f}(x)]^2 + Var[\hat{f}(x)] + \sigma^2$
where $Var[\hat{f}(x)] \propto \frac{1}{\sqrt{n}}$ (decreases with data)

**Use when**:
- **Large data**: Law of large numbers tames variance—with $n$ samples, variance decreases as $O(1/\sqrt{n})$
- **Ensemble available**: Bagging reduces variance by $\frac{1}{M}$ with $M$ uncorrelated models
- **Asymmetric costs**: Systematic bias (missing demographics) worse than random errors

**Solution**: Fraud detection with millions of transactions → use deep networks over logistic regression. Volume reduces variance, but no data fixes systematic bias.

---

**4. Why does shuffling matter during training?**

**Math**: SGD assumes mini-batches are i.i.d. samples from $P(X,y)$. Update rule:
$\theta_{t+1} = \theta_t - \eta \nabla_\theta \frac{1}{B} \sum_{i \in \text{batch}} L(f_\theta(x_i), y_i)$
Without shuffling, batches are temporally correlated → $Cov(\nabla_t, \nabla_{t+1}) \neq 0$ → biased updates

**Without shuffling**:
- Sequential batches → $E[\nabla_t | \nabla_{t-1}]$ correlated → oscillating convergence
- BatchNorm: $\mu_B, \sigma_B$ biased by temporal ordering
- Model learns sequence artifacts: $P(y|x, \text{position})$ instead of $P(y|x)$

**Solution**: Shuffle once per epoch. For time-series, use proper temporal splits (no shuffling). Check if gradient variance between epochs is high—indicates ordering issues.

---

**5. What if train/validation distributions differ?**

**Math**: Hyperparameter selection optimizes $\lambda^* = \arg\min_\lambda L_{val}(\theta_\lambda)$. But if $P_{val}(x,y) \neq P_{deploy}(x,y)$, then $\lambda^*_{val} \neq \lambda^*_{deploy}$

**Impact**:
- Hyperparameter selection unreliable: optimizing wrong objective
- Early stopping fails: stops at $\arg\min_t L_{val}(t)$ instead of $\arg\min_t L_{deploy}(t)$
- Performance estimates biased: $E_{P_{val}}[L] \neq E_{P_{deploy}}[L]$

**Solution**: Validation must mirror deployment. Use stratified splits. Check distribution stats (mean, std, quantiles). If validation uses recent/clean data, estimates will be overoptimistic.

---

### Optimization Behavior

**6. Why do gradients shrink while loss drops?**

**Math**: Near local minimum, first-order optimality: $\nabla_\theta L \to 0$. Gradient magnitude: $||\nabla L|| \approx ||\nabla^2 L|| \cdot ||s||$ where $s$ is distance to minimum (by Taylor expansion)

As we approach minimum: $||s|| \to 0$ → $||\nabla L|| \to 0$ even if loss still decreasing

**Normal**: Approaching minimum → flatter landscape (lower curvature) → smaller gradients. Flat minima correlate with better generalization (Hochreiter & Schmidhuber)

**Problem**:
- Vanishing gradients from saturated activations: $\sigma'(z) \to 0$ for $|z| \gg 0$ (sigmoid/tanh)
- Poor conditioning: $\kappa(H) = \frac{\lambda_{max}}{\lambda_{min}} \gg 1$ → some directions have tiny gradients

**Solution**: Plot $||\nabla \theta||$ over time. Smooth exponential decay = healthy. Sudden drop = check activation distributions (use ReLU: $\frac{\partial}{\partial z} \max(0,z) = 1$ for $z>0$), adjust learning rate, or verify loss still improving.

---

**7. What causes training instability after convergence?**

**Common causes**:
- Learning rate too high for refined optimization
- BatchNorm with small batches → noisy statistics
- Momentum overshooting good solutions
- Mixed precision accumulation errors

**Solution**: Use learning rate decay (cosine annealing). Apply gradient clipping (1.0-5.0). Switch to GroupNorm for small batches. Check FP16 loss scaling.

---

**8. Why does warm-up help large models?**

**Math**: Early gradients are noisy: $g_t = \nabla L + \epsilon_t$ where $||\epsilon_t|| \gg ||\nabla L||$ with random init.

Signal-to-noise ratio: $SNR = \frac{||\nabla L||}{||\epsilon_t||} \approx O(\frac{1}{\sqrt{d}})$ for $d$ parameters

With full learning rate: $||\Delta \theta|| = \eta ||g_t|| \approx \eta ||\epsilon_t||$ → large random updates

**Problem**: Full LR with noisy gradients → destabilizes training, explodes gradients (especially in deep networks where $||\nabla L^{(l)}|| = \prod_{i>l} ||J_i|| \cdot ||\nabla L^{(L)}||$), locks into poor minima

**Solution**: Warm-up linearly increases LR: $\eta_t = \eta_{max} \cdot \frac{t}{T_{warmup}}$ for $t \leq T_{warmup}$. Start small ($10^{-7}$), ramp over 1-10% of training. Critical for Adam and large batch sizes where noise compounds: effective gradient $\approx \frac{1}{B}\sum g_i$ has variance $\frac{\sigma^2}{B}$.

---

**9. Weight decay vs L2 regularization—what's the difference?**

**Math**:
- **L2 regularization**: $L_{total} = L(f_\theta(x), y) + \frac{\lambda}{2}||\theta||^2$, then compute $\nabla_\theta L_{total}$
- **Weight decay**: Update $\theta \leftarrow (1-\lambda)\theta - \eta \nabla_\theta L$

**For SGD**: Equivalent since $\nabla(L + \frac{\lambda}{2}||\theta||^2) = \nabla L + \lambda \theta$

**For Adam**: Different! Adam uses adaptive learning rates:
$m_t = \beta_1 m_{t-1} + (1-\beta_1)g_t, \quad v_t = \beta_2 v_{t-1} + (1-\beta_2)g_t^2$
$\theta_{t+1} = \theta_t - \eta \frac{m_t}{\sqrt{v_t} + \epsilon}$

With L2, gradient $g_t = \nabla L + \lambda \theta$ gets rescaled by $\frac{1}{\sqrt{v_t}}$ → heavily-updated params get less regularization

**Solution**: Use AdamW (Adam + weight decay): $\theta \leftarrow (1-\lambda)\theta - \eta \frac{m_t}{\sqrt{v_t}}$. Applies uniform regularization independent of adaptive scaling.

---

**10. Why do models learn spurious correlations faster?**

**Math**: SGD follows steepest descent with respect to loss $L$. Given two features $x_1$ (spurious), $x_2$ (causal), if $||\nabla_{x_1} L|| > ||\nabla_{x_2} L||$ initially, model learns $x_1$ first (simplicity bias / implicit regularization)

**Reasons**:
- Spurious patterns have simpler decision boundaries (lower VC dimension)
- Local features (texture) vs global (shape): $\nabla_{texture} L$ computable from small receptive fields
- Dataset artifacts highly predictive: $I(y; \text{artifact}) > I(y; \text{causal})$ in training data
- True causality requires multi-hop: $P(y|do(x))$ vs $P(y|x)$

**Solution**: Data augmentation to remove shortcuts (e.g., ColoredMNIST with random colors). Validate on out-of-distribution data where spurious correlations break. Use counterfactual examples. Check feature attribution (GradCAM, SHAP) to see what drives predictions.

---

### Neural Network Internals

**11. Why do deep networks need normalization?**

**Math**: Internal covariate shift—layer inputs change distribution as params update

**Problems**: Each layer adapts to moving targets, gradients vanish/explode exponentially, learning rate hard to tune (different scales per layer)

**Solution**: BatchNorm stabilizes distributions, enables higher learning rates, acts as implicit regularization. Use LayerNorm for small batches or sequential data. GroupNorm for very small batches.

---

**12. What causes dead neurons (ReLU)?**

**Math**: ReLU activation $a = \max(0, z)$ where $z = w^Tx + b$.

Gradient: $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial a} \cdot \frac{\partial a}{\partial z} \cdot x = \frac{\partial L}{\partial a} \cdot \mathbb{1}_{z>0} \cdot x$

If $z = w^Tx + b < 0$ for all inputs → $\frac{\partial a}{\partial z} = 0$ always → no gradient → stuck permanently (dying ReLU)

**Causes**:
- Large negative bias initialization
- Aggressive learning rate: update $\Delta w = -\eta g$ pushes $w$ such that $w^Tx + b < 0$ for all $x$ in data manifold
- Once dead, $\nabla w = 0$ → no recovery mechanism

**Solution**: Use LeakyReLU ($a = \max(0.01z, z)$) or PReLU ($a = \max(\alpha z, z)$ where $\alpha$ is learned). Apply He initialization: $w \sim N(0, \frac{2}{n_{in}})$. Reduce LR. Monitor dead neuron fraction: $\frac{1}{N}\sum_{i=1}^N \mathbb{1}_{a_i = 0}$ during training.

---

**13. Why does initialization matter more for deep models?**

**Math**: Forward pass variance propagation. For layer $l$:
$y^{(l)} = W^{(l)} x^{(l-1)} + b^{(l)}$
$Var(y^{(l)}_i) = n_{l-1} \cdot Var(w^{(l)}_{ij}) \cdot Var(x^{(l-1)}_j)$

If $Var(y^{(l)}) = c \cdot Var(x^{(l-1)})$, then after $L$ layers: $Var(y^{(L)}) = c^L \cdot Var(x^{(0)})$
- If $c > 1$: activations explode ($c^L \to \infty$)
- If $c < 1$: activations vanish ($c^L \to 0$)

Backward pass: $\frac{\partial L}{\partial x^{(l)}} = (W^{(l+1)})^T \frac{\partial L}{\partial x^{(l+1)}}$ → same exponential effect on gradients

**Impact**: Poor init → signals/gradients unusable. Symmetry breaking required—identical weights → identical gradients → neurons never differentiate

**Solution**:
- Xavier: $w \sim N(0, \frac{2}{n_{in} + n_{out}})$ for tanh/sigmoid (maintains $Var = 1$)
- He: $w \sim N(0, \frac{2}{n_{in}})$ for ReLU (accounts for half neurons dead)

---

**14. Why does dropout sometimes hurt?**

**Use dropout when**: Model overfits, have sufficient capacity, training for many epochs

**Skip dropout when**:
- Model already simple/small (underfitting)
- Using BatchNorm (they interact poorly—use one or other)
- Small dataset where model needs full capacity
- Insufficient training time for ensemble effect

**Solution**: Modern architectures often skip dropout in favor of other regularization (weight decay, data augmentation, BatchNorm). Test with/without on validation set.

---

**15. How do residual connections help?**

**Math**: $y = F(x) + x$ creates gradient highway: $∂L/∂x = ∂L/∂y · (∂F/∂x + I)$

**Benefits**:
- Gradient flows directly to early layers (prevents vanishing)
- Model learns incremental refinements (identity + Δ) not full transformations
- Creates ensemble of different-depth sub-networks
- Smooths loss landscape

**Solution**: Use ResNet-style skip connections for ```networks >20``` layers. Enables training ```50-200+ layer networks``` that would otherwise be impossible.

---

### Practical Debugging

**16. How debug high-confidence but wrong predictions?**

**Math**: Calibration issue—P(correct|confidence) ≠ confidence

**Check**:
- Dataset leakage or train/test contamination
- Spurious correlation exploitation (visualize high-confidence errors)
- Label quality for those examples
- Overfitting to specific patterns

**Solution**: Apply temperature scaling post-training. Use ensembles or MC dropout for uncertainty. Examine attention/activations on confident mistakes. Check reliability diagrams (calibration plots).

---

**17. Why does adding features sometimes hurt performance?**

**Math**: Curse of dimensionality. In $d$-dimensional space with $n$ samples:
- Volume of hypercube: $V \propto r^d$ → grows exponentially
- Sample density: $\rho = \frac{n}{r^d} \propto \frac{n}{r^d}$ → for fixed $n$, $\rho \to 0$ as $d$ grows
- Distance concentration: $\frac{d_{max} - d_{min}}{d_{min}} \to 0$ as $d \to \infty$ (all points equidistant)

**Problems**:
- Sparse coverage: need $n = O(r^d)$ samples to maintain density (exponential in $d$)
- Noise features dilute SNR: $SNR = \frac{\text{signal variance}}{\text{noise variance}} = \frac{k \sigma_s^2}{(d-k)\sigma_n^2}$ where $k$ is informative features
- Spurious correlations: $O(d^2)$ pairwise correlations, many random
- Overfitting easier: VC dimension grows with features

**Solution**: Feature selection (Lasso: $\min_w ||y - Xw||^2 + \lambda ||w||_1$, tree-based importance). Dimensionality reduction (PCA: $X_{reduced} = XV_k$ where $V_k$ are top $k$ eigenvectors). Or collect more data: $n \gg d$.

---

**18. Underfitting vs noisy data—how to tell?**

**Underfitting signs**:
- Train loss high and plateaued
- Both train and val poor
- Increasing capacity helps
- Training longer doesn't help

**Noisy data signs**:
- Train loss decreases but val plateaus
- Individual examples inconsistent
- Human performance poor on task
- Cleaning/relabeling subset significantly improves val

**Solution**: Check Bayes error (human performance). If high → noisy data. If low but model poor → underfitting. Measure annotator agreement.

---

**19. How validate preprocessing pipeline?**

**Critical checks**:
- Visualize transformed samples—sensible?
- Apply inverse transform → verify reconstruction
- Check statistics (mean, std, range) at each stage
- Test edge cases (empty, outliers, missing values)
- Compare train vs val distributions post-preprocessing
- Train simple model—if can't learn anything, preprocessing likely broken

**Solution**: Unit test each transformation. Save before/after examples. Create synthetic test cases with known outputs.

---

**20. Why does training differ on GPU vs CPU?**

**Causes**:
- Precision differences (FP16 vs FP32)
- Non-deterministic operations (cuDNN algorithms)
- Parallel accumulation order varies
- Memory layout differences

**Solution**: Set deterministic flags and seeds, but expect small numerical differences. Large divergence indicates instability—check mixed precision settings, gradient clipping, or condition number.

---


## QnA -2
## 🟡 P1 – LLMs, Generative Models & Failure Modes




### Model Behavior & Limitations

**21. Why do LLMs struggle with multi-step reasoning?**

**Math**: Single forward pass generates tokens autoregressively: ```P(y_t | y_<t,x)```

**Limitations**:
- No iteration or backtracking—each token independent
- No working memory for intermediate steps
- Errors compound—one wrong step derails everything
- Pattern matching, not symbolic logic execution
- Long reasoning chains exceed effective context window

**Solution**: Prompt for explicit chain-of-thought. Use tool-calling for computation (calculator, code execution). Implement multi-turn refinement. Break complex reasoning into smaller steps.

---

**22. What causes repetition loops?**

**Sampling trapped in high-probability cycles**: ```p(token | context_with_repetition) → same token → reinforces pattern```

**Triggers**:
- Temperature too low → deterministic sampling
- Context fills with pattern → attention focuses on it
- Training data contained repetitive patterns

**Solution**: Apply repetition penalty (penalize recently used tokens). Increase temperature slightly. Use nucleus sampling (top-p) instead of greedy. Set presence/frequency penalties to break cycles.

---

**23. Why do LLMs ignore system prompts?**

**Competition**: System prompts compete with strong training priors

**Reasons**:
- Training data patterns overpower weak system instructions
- Instruction-tuning didn't cover diverse system prompt formats
- User content more salient than system context
- System prompts lost in long conversations

**Solution**: Make system prompts specific and concise. Reinforce key constraints in user messages. Use few-shot examples. Keep conversations focused to maintain system context.

---

**24. How does sampling affect diversity vs correctness?**

**Math**: Sample from $P(token|context)$ with temperature $\tau$:
$P_\tau(w_i) = \frac{\exp(z_i/\tau)}{\sum_j \exp(z_j/\tau)}$
where $z_i$ are logits. As $\tau \to 0$: $P_\tau \to$ one-hot (argmax). As $\tau \to \infty$: $P_\tau \to$ uniform

**Strategies**:
- **Low temp ($\tau = 0.1-0.3$)**: Sharpens distribution → high correctness, low diversity (deterministic argmax)
- **High temp ($\tau = 0.8-1.2$)**: Flattens distribution → high diversity, lower correctness (samples unlikely tokens)
- **Top-p (nucleus)**: Sample from smallest set $V$ where $\sum_{w \in V} P(w) \geq p$. Adaptive cutoff
- **Top-k**: Sample from top $k$ highest probability tokens

Expected entropy: $H = -\sum_i P_\tau(w_i) \log P_\tau(w_i)$ increases with $\tau$

**Solution**: Factual tasks → low temp. Creative tasks → higher temp. Use nucleus ($p=0.9$) for balance. Beam search (keep top-$k$ sequences at each step) for coherence but lower diversity.

---

**25. When doesn't scaling help downstream performance?**

**Reasons**:
- Task doesn't require emergent capabilities
- Larger models overfit to pretraining biases
- Downstream data too small (overfits)
- Evaluation measures superficial patterns small models capture
- Larger models need more careful prompting

**Solution**: Right-size models to tasks. Improve prompting before scaling. Test if task benefits from scale (reasoning vs pattern matching). Consider efficiency—small tuned model often beats large zero-shot.

---

### Context & Memory

**26. Why do LLMs "forget" long context?**

**Math**: Self-attention computes attention weights via softmax over sequence length $N$:
$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$
$\alpha_{ij} = \frac{\exp(q_i \cdot k_j / \sqrt{d_k})}{\sum_{j'=1}^N \exp(q_i \cdot k_{j'} / \sqrt{d_k})}$

As $N$ grows large: $\alpha_{ij} \approx \frac{1}{N}$ (each token gets equal weight)

**Problems**:
- Attention dilution: Individual important tokens receive weight $\approx \frac{1}{N}$ → signal lost in noise
- Positional encoding degrades: Sinusoidal or learned positions extrapolate poorly beyond training length
- Training bias: Most training sequences shorter → long-range dependencies undertrained
- Gradient flow: $\frac{\partial L}{\partial x_1} = \frac{\partial L}{\partial x_N} \cdot \prod_{l} J_l$ where $J_l$ involves attention → weakens with distance
- Computational: $O(N^2)$ memory and compute for attention matrix

**Solution**: Summarize important info periodically. Use retrieval (embed + search top-$k$ relevant segments). Re-inject critical information. Sparse/hierarchical attention patterns. Extend positional encodings (RoPE, ALiBi).

---

**27. What are failure modes of long-context LLMs?**

**Common issues**:
- **Lost in the middle**: Ignores middle portions (primacy/recency bias)
- **Attention dilution**: Each token gets ≈1/N attention weight
- **Conflicting information**: Can't resolve contradictions in long docs
- **Fabrication**: Relevant info present but not attended to
- **Increased latency/cost**: Quadratic attention complexity

**Solution**: Test with needle-in-haystack. Use retrieval to focus on relevant segments. Prompt to consider entire context. Monitor position-dependent accuracy.

---

**28. Why is retrieval better than just increasing context?**

**Math**: Attention complexity O(N²), retrieval O(N log N) with index

**Retrieval advantages**:
- Focuses attention on relevant information only
- Scales better than quadratic attention
- Can leverage external, updatable knowledge
- Reduces cost (process less tokens)
- More interpretable—see what was retrieved

**Solution**: Hybrid approach works best: retrieve top-k relevant segments, then reason over them within context window. Embed query + docs, retrieve by similarity, rerank by relevance.

---

### Alignment & Instruction Following

**29. Why does RLHF reduce creativity?**

**Math**: RLHF optimizes policy $\pi_\theta$ via reward model $r_\phi$:
$\max_\theta \mathbb{E}_{x \sim P(x), y \sim \pi_\theta(y|x)}[r_\phi(x,y)] - \beta \cdot D_{KL}(\pi_\theta || \pi_{ref})$

where $\pi_{ref}$ is the pretrained base model, $\beta$ controls how much we deviate

**Effect**:
- Reward model $r_\phi$ trained on human preferences → biased toward safe, conventional responses (mode-seeking)
- KL constraint $D_{KL}(\pi_\theta || \pi_{ref}) = \mathbb{E}_{y \sim \pi_\theta}[\log \pi_\theta(y|x) - \log \pi_{ref}(y|x)]$ penalizes deviation from base model
- Mode collapse: $\pi_\theta$ concentrates on high-reward modes, ignoring creative but risky options
- Human raters reward "helpful-looking" verbose responses → policy learns superficial markers

**Why creativity suffers**:
- Creative responses have higher variance in human ratings → lower expected reward
- Novel outputs penalized by KL term (weren't in $\pi_{ref}$ distribution)
- Risk-averse training: prefer consistent 7/10 over occasional 10/10 or 3/10

**Solution**: Adjust KL penalty $\beta$ (lower $\beta$ = more exploration). Use DPO (Direct Preference Optimization) which optimizes directly on preferences. Prompt explicitly for creativity. Use base models for creative generation.

---

**30. How does alignment introduce new biases?**

**Sources**:
- Demographic skew in rater populations
- Cultural preferences in "helpful" ratings
- Overcompensation creating different biases
- Reward hacking—model learns superficial markers
- Underrepresentation of certain use cases

**Solution**: Diverse rater pool. Red-teaming for edge cases. Monitor fairness metrics across demographics. Continuous evaluation on real use cases. Biases shift, don't disappear.

---

**31. Why instruction-tuned models worse on raw text?**

**Math**: Distribution shift—model trained on $P(y|instruction, x)$, deployed on $P(y|x)$

**Problems**:
- Expects structured prompts, not raw completion
- Training emphasized Q&A format
- Safety tuning causes refusals on ambiguous inputs
- Loss of raw language modeling capability

**Solution**: Use base models for completion tasks. Use instruct models for interactive tasks. Fine-tune on your specific use case if distribution differs significantly.

---

**32. Safety vs usefulness tradeoffs?**

**Tighter safety**: More false refusals, reduced nuance handling, over-cautious responses

**Looser safety**: Potential misuse, harmful content generation, liability issues

**Solution**: Precise safety that doesn't overbroadly restrict. Iterative refinement based on real failures. Context-aware safety (medical discussion ≠ instructions to harm). User feedback loops.

---

**33. Why does instruction following fail on OOD tasks?**

**Coverage limitation**: Instruction-tuning data is finite

**Causes**:
- Novel task formats not in training
- Unusual phrasing confuses instruction parsing
- Domain-specific jargon not covered
- Complex instructions exceed training complexity

**Solution**: Use few-shot examples to bridge gap. Provide clearer, more explicit instructions. Decompose complex tasks into familiar subtasks. Fine-tune on domain-specific instruction data.

---

### Evaluation & Trust

**34. Why is "looks correct" dangerous?**

**LLMs are persuasive ≠ accurate**:
- Confident tone doesn't mean correctness
- Plausible fabrications (hallucinations) common
- Evaluator expertise matters—non-experts miss subtle errors
- Confirmation bias—see what you expect

**Solution**: Verify factual claims against sources. Use automated checks for objective criteria. Expert review for domain content. Cross-reference multiple sources. Check reasoning steps, not just final answer.

---

**35. How detect benchmark over-optimization?**

**Signs**:
- Benchmark scores improve but real-world performance doesn't
- Poor performance on slight benchmark variations
- Exploits dataset artifacts (spurious correlations)
- Training loss decreases but validation plateaus

**Solution**: Hold-out test sets. Use multiple diverse benchmarks. Test adversarial examples. Evaluate on actual use cases. Never train on test distribution. Check if improvements transfer to related tasks.

---

**36. Why do human evaluators disagree on LLM outputs?**

**Subjectivity**: Quality is multifaceted—helpfulness, accuracy, style, tone

**Causes**:
- Task ambiguity—unclear "good" definition
- Personal preferences vary
- Context-dependent judgments
- Evaluator fatigue and attention issues

**Solution**: Multiple raters per example. Clear rubrics with examples. Pairwise comparisons (easier than absolute ratings). Measure inter-rater agreement (Cohen's kappa). Expert calibration sessions.

---

## QnA -3
## 🔵 P2 – Production & Systems



### Infrastructure & Scaling

**37. Why does cost grow non-linearly with traffic?**

**Step functions in infrastructure**:
- Need new servers/regions at thresholds
- Load balancers, redundancy complexity
- Retrieval index size grows → search slower
- Cache hit rates decrease with query diversity
- Cross-region replication for reliability
- More edge cases need handling

**Solution**: Plan for step functions. Use auto-scaling with buffers. Optimize cache strategies. Monitor cost per request. Consider reserved capacity for baseline load.

---

**38. When is horizontal scaling ineffective?**

**Horizontal scaling (more servers) fails when**:
- Latency dominated by single-request processing, not throughput
- Model too large for single GPU—needs model parallelism
- Cross-server communication overhead dominates
- Stateful operations require sticky routing (caching, sessions)

**Solution**: Model optimization (quantization, distillation). Batch requests efficiently. Vertical scaling with bigger GPUs. Hybrid approach—horizontal for throughput, vertical for latency.

---

**39. Why does throughput optimization increase errors?**

**Quality tradeoffs**:
- Larger batches change behavior (BatchNorm effects)
- Aggressive quantization loses precision
- Reduced context for speed misses information
- Caching serves stale results
- Faster/smaller models less capable

**Solution**: Monitor error rates alongside throughput. Set quality thresholds (SLAs). A/B test optimizations. Don't optimize past acceptable error rate. Use graceful degradation.

---

**40. CPU vs GPU inference—how to choose?**

**GPUs win for**:
- Large models benefiting from parallelism
- High throughput (batch many requests)
- Low latency where GPU speed critical

**CPUs win for**:
- Small models (GPU overhead not worth it)
- Low query volume (GPU sits idle)
- Cost sensitivity at low utilization
- Easier deployment/flexibility

**Solution**: Test both. Calculate cost per request, not just raw speed. Consider utilization—idle GPU wastes money. Use GPUs for batch workloads, CPUs for sporadic queries.

---

**41. Latency SLAs vs answer quality tradeoffs?**

**Tighter latency requires**:
- Smaller models (less capable)
- Shorter contexts (less information)
- Fewer retrieval candidates (lower recall)
- Less generation time (shorter answers)
- Aggressive caching (staleness risk)

**Looser latency allows**:
- Better models
- Comprehensive retrieval
- Iterative refinement
- Multi-stage processing

**Solution**: Define acceptable latency per use case. Different queries justify different SLAs (complex queries → slower OK). Use tiered service (fast path for simple, slow path for complex). Optimize within constraints, not blindly.